In [ ]:
def simulate_from_raw_only_W(real_data, s2gxg=0.9, s2e=0.1):
    Z = (real_data - real_data.mean(axis=0)) / real_data.std(axis=0)
    n, m = Z.shape
    KZ = Z @ Z.T
    D = Z * Z
    p = m * (m - 1) / 2
    W = 0.5 * (KZ * KZ - D @ D.T) / p
    cov_y = s2gxg * W + s2e * np.eye(n)
    y = np.random.multivariate_normal(mean=np.zeros(n), cov=cov_y)
    y -= np.mean(y)  
    return Z, y

def MoM_only_M(Z, y, nmc=40):
    n = Z.shape[0]
    m = Z.shape[1]
    p = m*(m-1)/2
    u = np.random.randn(n,nmc)
    ZZ = Z*Z
    
    Du = ZZ @ (ZZ.T @ u) 

    KKu = np.zeros((n, nmc))
    for i in range(nmc):
        Z_tilde = u[:, i].reshape(-1, 1) * Z   
        M = Z.T @ Z_tilde
        ZM = Z @ M
        KKu[:, i] = np.sum(ZM * Z, axis=1)

    ###trW_app
    trW_app = 0.5*(np.mean(np.sum(u * KKu, axis=0)) - np.mean(np.sum(u * Du, axis=0)))/p
    trKK2 =np.mean(np.sum(KKu *KKu, axis=0))
    trDK =np.mean(np.sum(Du *KKu, axis=0))
    trDD = np.mean(np.sum(Du * Du, axis=0))
    trW2_app = (trKK2 - 2*trDK +trDD)/(4*p**2)
    ############# Quadratic forms part
    yTy = y @ y
    yTWy = 0.5*(y.T@(np.sum((Z @(Z.T@(y.reshape(-1, 1) * Z)))*Z,axis = 1)) - (y @ (ZZ @ (ZZ.T @ y))))/p

    A = np.array([
        [trW2_app, trW_app],
        [trW_app, n]
    ])
    
    b = np.array([
        yTWy,
        yTy
    ]).reshape(-1, 1)
     
    x1 = np.linalg.solve(A, b)
    return x1[0],x1[1],A